# Table of Contents
 <p><div class="lev1"><a href="#Import-Python-modules"><span class="toc-item-num">1 - </span>Import <code>Python</code> modules</a></div><div class="lev1"><a href="#Set-CONSTANTS"><span class="toc-item-num">2 - </span>Set <em>CONSTANTS</em></a></div><div class="lev1"><a href="#Create-Valuation-Model"><span class="toc-item-num">3 - </span>Create Valuation Model</a></div><div class="lev1"><a href="#Import-Data"><span class="toc-item-num">4 - </span>Import Data</a></div><div class="lev1"><a href="#Set-Outputs-to-Display"><span class="toc-item-num">5 - </span>Set Outputs to Display</a></div><div class="lev1"><a href="#Run-Valuation-Model"><span class="toc-item-num">6 - </span>Run Valuation Model</a></div>

# Import `Python` modules

In [1]:
from __future__ import print_function
import pandas

from CorpFin.Valuation import ValuationModel
from HelpyFuncs.Print import print_live_in_ipython

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


In [2]:
print_live_in_ipython()

# Set _CONSTANTS_

In [3]:
# Year 0
YEAR_0 = 1980

# Number of Pro-Forma Years EXCLUDING Year 0
NB_PRO_FORMA_YEARS = 3

# U.S. Corporate Tax rate
US_CORP_TAX_RATE = .48

# Risk-Free Rate
RISK_FREE_RATE = .11

# Public Market Return
PUB_MKT_RETURN = .19

# CAPM Betas
PUB_MKT_BETA_FOR_FIN_SECTOR = 1.   # assumed in line with overall equity market
PEVC_BETA_FOR_PRO_FORMA_PERIOD = 2.

# Long-Term Growth Rate
LONG_TERM_GROWTH_RATE = .08

# Create Valuation Model

In [4]:
model = \
    ValuationModel(
        venture_name='TechnicalData',
        year_0=YEAR_0,
        nb_pro_forma_years_excl_0=NB_PRO_FORMA_YEARS)#,
        # compile=False)

Compiling:
    PublicMarketPremium
    ProFormaPeriodAssetDiscountRate
    ProFormaPeriodDiscountRate
    StabilizedDiscountRate
    Revenue
    RevenueChange
    RevenueGrowth
    OpEx
    OpEx_over_Revenue
    OpExGrowth
    EBIT
    EBITMargin
    EBITGrowth
    EBIAT
    FA
    FA_over_Revenue
    FAGrowth
    Depreciation
    Depreciation_over_prevFA
    CapEx
    CapEx_over_Revenue
    CapEx_over_RevenueChange
    CapExGrowth
    NWC
    NWC_over_Revenue
    NWCGrowth
    NWCChange
    NWCChange_over_Revenue
    NWCChange_over_RevenueChange
    FCF
    TV
    TV_RevenueMultiple
    TV_EBITMultiple
    Val_of_FCF
    Val_of_TV
    Unlev_Val
    DERatios
    Debt
    InterestRates
    InterestExpense
    ITS
    ProFormaPeriodITSDiscountRate
    StabilizedITSDiscountRate
    ITS_TV
    Val_of_ITS
    Val_of_ITS_TV
    Val
done!


# Import Data

In [5]:
data_df = \
    pandas.read_csv(
        'data/Technical Data.csv',
        index_col='Year')

data_df.T

Year,1981,1982,1983
Revenue,288000,799350,1181700
OpEx,242774,308991,410010
extra_executive_compensation,2500,75000,140000


# Set Outputs to Display

In [6]:
outputs = \
    ['RiskFreeRate', 'PublicMarketReturn', 'PublicMarketPremium',
     ' ',
     'ProFormaPeriodBeta', 'ProFormaPeriodDiscountRate',
     '  ',
     'StabilizedBeta', 'StabilizedDiscountRate',
     '   ',
     'REVENUE calcs:', 'Revenue', 'RevenueGrowth',
     '    ',
     'OPEX calcs:', 'OpEx', 'OpExGrowth',
     '     ',
     'EBIT calcs:', 'EBIT', 'EBITMargin', 'EBITGrowth',
     '      ',
     'EBIAT calcs:', 'CorpTaxRate', 'EBIAT',
     '       ',
     'FCF & TV calcs:', 'FCF', 'LongTermGrowthRate', 'TV',
     '        ',
     'VALUATION calcs:', 'Val_of_FCF', 'Val_of_TV', 'Val']

# Run Valuation Model

In [7]:
results = \
    model(
        outputs=outputs,
        Revenue=[0.] + data_df.Revenue.tolist(),
        OpEx=[0.] + map(lambda x: x[0] + x[1],
                        zip(data_df.OpEx, data_df.extra_executive_compensation)),
        CorpTaxRate=US_CORP_TAX_RATE,
        RiskFreeRate=RISK_FREE_RATE,
        PublicMarketReturn=PUB_MKT_RETURN,
        ProFormaPeriodBeta=PEVC_BETA_FOR_PRO_FORMA_PERIOD,
        StabilizedBeta=PUB_MKT_BETA_FOR_FIN_SECTOR,
        LongTermGrowthRate=LONG_TERM_GROWTH_RATE)
    
pandas.options.display.float_format = '{:,.2f}'.format
results['data_frame'].T

Calculating:
    PublicMarketPremium
    ProFormaPeriodDiscountRate
    StabilizedDiscountRate
    Revenue
    RevenueGrowth
    OpEx
    OpExGrowth
    EBIT
    EBITMargin
    EBITGrowth
    EBIAT
    FCF
    TV
    Val_of_FCF
    Val_of_TV
    Val
done!


,Year 0,1981,1982,1983
RiskFreeRate,0.11,,,
PublicMarketReturn,0.19,,,
PublicMarketPremium,0.08,,,
,,,,
ProFormaPeriodBeta,2.00,,,
ProFormaPeriodDiscountRate,0.27,,,
,,,,
StabilizedBeta,1.00,,,
StabilizedDiscountRate,0.19,,,
,,,,
